# Run: rdrp1 Genome/Exome sequencing (mammal)

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 21
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210121_rWgs/
```

### Intro/Objectives

> There are no separate things in the physical world, either. The physical world is wiggly. Clouds, mountains, trees, people, are all wiggly. And only when human beings get to working on things--they build buildings in straight lines, and try to make out that the world isn't really wiggly. But here we are, sitting in this room all built out of straight lines, but each one of us is as wiggly as all get-out.

Searching for RNA viruses in DNA-sequencing data...

### WGS (mammalian, non-mouse)

- File: `wgs_SraRunInfo.csv`

Search Term:
```
("Mammalia"[Organism] NOT "Mus musculus"[orgn]) AND ("type_exome"[Filter] OR "type_genome"[Filter]) NOT "METAGENOME" AND "platform illumina"[Properties] AND cluster_public[prop]
```
SRA Accessed: `2021/01/21`

Results: `330 239 / 393 278`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Thu Jan 21 12:43:54 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [3]:
cd $WORK

# Copy SRA Run Info
ln -s ../201230_sra/wgs_SraRunInfo.csv ./

md5sum wgs*
ls -alh wgs*

ln: failed to create symbolic link './wgs_SraRunInfo.csv': File exists
47313cfe06b2fce2ada17a894a72a278  wgs_SraRunInfo.csv
lrwxrwxrwx 1 artem artem 32 Jan 21 12:45 wgs_SraRunInfo.csv -> ../201230_sra/wgs_SraRunInfo.csv


In [6]:
tail -n+2 wgs_SraRunInfo.csv | shuf - > wgs.tmp

head -n 1 wgs_SraRunInfo.csv > wgs_1_SraRunInfo.csv
head -n 175000 wgs.tmp      >> wgs_1_SraRunInfo.csv

head -n 1 wgs_SraRunInfo.csv > wgs_2_SraRunInfo.csv
tail -n+175000 wgs.tmp      >> wgs_2_SraRunInfo.csv

rm wgs.tmp

In [ ]:
#aws s3 sync ./ $S3_WORK/

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [21]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Connection to ec2-3-208-57-35.compute-1.amazonaws.com closed by remote host.
diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..e4fc917 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49

   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
c

module.download.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-dl]
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-dl-20210122082930722800000004]
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.monitoring.aws_iam_role_policy_attachment.attachment: Creation complete after 1s [id=SerratusIamRole-monitor-20210122082930910900000005]
module.monitoring.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-monitor:CloudwatchGetMetrics]
module.scheduler.aws_iam_role_policy.scheduler: Creation complete after 1s [id=SerratusIamRole-scheduler:DescribeInstances-scheduler]
module.align.aws_iam_role_policy.ec2Terminate: Creating.

module.monitoring.aws_eip.monitor: Creating...
module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-07d966d99a6e62d90]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-093a4e6971c4f9174]
local_file.hosts: Creating...
local_file.create_tunnel: Creating...
module.merge.aws_launch_configuration.worker: Creating...
local_file.create_tunnel: Creation complete after 0s [id=9a387b991b591cda41a18209bfe30128feebfd5b]
module.align.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=ef00388df4913a9e5b75d7b3adae3ed3e84

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3200,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1350,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 25,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [22]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [16]:
# BATCH 1
cd $WORK
BATCH1='wgs_1_SraRunInfo.csv'
wc -l $WORK/$BATCH1

175001 /home/artem/serratus/notebook/210111_ab/wgs_1_SraRunInfo.csv


In [17]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH1

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/wgs_1_SraRunInfo.csv
  date: Thu Jan 21 13:23:47 PST 2021
  wc  : 175001 /home/artem/serratus/notebook/210111_ab/wgs_1_SraRunInfo.csv
  md5 : f20598c51b01785aa28a7744c0e0d129  /home/artem/serratus/notebook/210111_ab/wgs_1_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
63e4820c945370e11d21609bd7784ed8  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
a7f8aba288d55613464834819bb62500  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
38f57c1b6109d0831e419f1f07b007f1  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
39648e2f6f345c2b519001cc6e34a0e5  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

In [24]:
# BATCH 2
cd $WORK
BATCH2='wgs_2_SraRunInfo.csv'
wc -l $WORK/$BATCH2

218279 /home/artem/serratus/notebook/210111_ab/wgs_2_SraRunInfo.csv


In [25]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH2

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/wgs_2_SraRunInfo.csv
  date: Fri Jan 22 00:40:12 PST 2021
  wc  : 218279 /home/artem/serratus/notebook/210111_ab/wgs_2_SraRunInfo.csv
  md5 : a271a34380cea0f560935f091a8dc2b6  /home/artem/serratus/notebook/210111_ab/wgs_2_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
90e8bfb2ae1137a7ba83848d61c44014  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
cce05d1964c42da404cbfc9a19b9f737  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
d52abcb8033bbbfe53f2158e528965db  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
ffde292a0274d8858fd09a3c4a3b63b1  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

## Run Serratus

In [26]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

client_loop: send disconnect: Broken pipe
  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 35,
  "ALIGN_SCALING_CONSTANT": 0.1,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 2250,
  "CLEAR_INTERVAL": 300,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1200,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 35,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 35,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":35,"ALIGN_SCALING_CONSTANT":0.1,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":2250,"CLEAR_INTERVAL":300,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":1200,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":35,"MERGE_SCALING_CONSTANT":0.1,"MERGE_SCALING_ENAB

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
